In [ ]:
#import nemo
import nemo.collections.asr as nemo_asr

In [13]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained("nvidia/speakerverification_en_titanet_large")

[NeMo W 2024-11-20 13:49:14 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-11-20 13:49:14 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data

[NeMo I 2024-11-20 13:49:14 nemo_logging:381] PADDING: 16


[NeMo W 2024-11-20 13:49:15 nemo_logging:393] /Users/gsp/miniconda3/envs/paligemma/lib/python3.12/site-packages/nemo/core/connectors/save_restore_connector.py:682: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues rel

[NeMo I 2024-11-20 13:49:15 nemo_logging:381] Model EncDecSpeakerLabelModel was successfully restored from /Users/gsp/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/0dc382f40121a5fbd34db10a2bb04d826c2be6a8/speakerverification_en_titanet_large.nemo.


In [14]:
print(type(speaker_model))

<class 'nemo.collections.asr.models.label_models.EncDecSpeakerLabelModel'>


In [23]:
speaker_model.verify_speakers("manual_data/proc/a1.wav","manual_data/proc/a2.wav")

[NeMo I 2024-11-20 14:50:26 nemo_logging:381]  two audio files are from same speaker


True

In [24]:
import torch

In [35]:
import numpy as np
#import torch
from lightning.pytorch import seed_everything
from omegaconf import OmegaConf
from nemo.collections.asr.data.audio_to_label import AudioToSpeechLabelDataset
from nemo.collections.asr.models import EncDecSpeakerLabelModel
from nemo.collections.asr.parts.features import WaveformFeaturizer
from nemo.core.config import hydra_runner
from nemo.utils import logging
seed_everything(42)

print("hello")

Seed set to 42


hello


In [36]:
def main_g(cfg_path="conf/speaker_identification_infer.yaml"):
    cfg = OmegaConf.load(cfg_path)
    logging.info(f'Hydra config: {OmegaConf.to_yaml(cfg)}')

    device = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'

    enrollment_manifest = cfg.data.enrollment_manifest
    test_manifest = cfg.data.test_manifest
    out_manifest = cfg.data.out_manifest
    sample_rate = cfg.data.sample_rate

    backend = cfg.backend.backend_model.lower()

    featurizer = WaveformFeaturizer(sample_rate=sample_rate)
    dataset = AudioToSpeechLabelDataset(manifest_filepath=enrollment_manifest, labels=None, featurizer=featurizer)
    enroll_id2label = dataset.id2label

    if backend == 'cosine_similarity':
        model_path = cfg.backend.cosine_similarity.model_path
        batch_size = cfg.backend.cosine_similarity.batch_size
        if model_path.endswith('.nemo'):
            speaker_model = EncDecSpeakerLabelModel.restore_from(model_path)
        else:
            speaker_model = EncDecSpeakerLabelModel.from_pretrained(model_path)

        enroll_embs, _, enroll_truelabels, _ = speaker_model.batch_inference(
            enrollment_manifest,
            batch_size,
            sample_rate,
            device=device,
        )

        test_embs, _, _, _ = speaker_model.batch_inference(
            test_manifest,
            batch_size,
            sample_rate,
            device=device,
        )

        # length normalize
        enroll_embs = enroll_embs / (np.linalg.norm(enroll_embs, ord=2, axis=-1, keepdims=True))
        test_embs = test_embs / (np.linalg.norm(test_embs, ord=2, axis=-1, keepdims=True))

        # reference embedding
        reference_embs = []
        keyslist = list(enroll_id2label.values())
        for label_id in keyslist:
            indices = np.where(enroll_truelabels == label_id)
            embedding = (enroll_embs[indices].sum(axis=0).squeeze()) / len(indices)
            reference_embs.append(embedding)

        reference_embs = np.asarray(reference_embs)

        scores = np.matmul(test_embs, reference_embs.T)
        matched_labels = scores.argmax(axis=-1)

    elif backend == 'neural_classifier':
        model_path = cfg.backend.neural_classifier.model_path
        batch_size = cfg.backend.neural_classifier.batch_size

        if model_path.endswith('.nemo'):
            speaker_model = EncDecSpeakerLabelModel.restore_from(model_path)
        else:
            speaker_model = EncDecSpeakerLabelModel.from_pretrained(model_path)

        if speaker_model.decoder.final.out_features != len(enroll_id2label):
            raise ValueError(
                "number of labels mis match. Make sure you trained or finetuned neural classifier with labels from enrollement manifest_filepath"
            )

        _, test_logits, _, _ = speaker_model.batch_inference(
            test_manifest,
            batch_size,
            sample_rate,
            device=device,
        )
        matched_labels = test_logits.argmax(axis=-1)

    with open(test_manifest, 'rb') as f1, open(out_manifest, 'w', encoding='utf-8') as f2:
        lines = f1.readlines()
        for idx, line in enumerate(lines):
            line = line.strip()
            item = json.loads(line)
            item['infer'] = enroll_id2label[matched_labels[idx]]
            json.dump(item, f2)
            f2.write('\n')

    logging.info("Inference labels have been written to {} manifest file".format(out_manifest))

In [47]:
main_g()

[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Hydra config: name: SpeakerIdentificationInfer
    data:
      enrollment_manifest: ./content/data/an4/wav/an4_clstk/train_all_manifest.json
      test_manifest: ./content/data/an4/wav/an4test_clstk/test_all_manifest.json
      out_manifest: ./infer_output.json
      sample_rate: 16000
    backend:
      backend_model: cosine_similarity
      cosine_similarity:
        model_path: titanet_large
        batch_size: 32
      neural_classifier:
        model_path: ???
        batch_size: 32
    


[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Dataset successfully loaded with 948 items and total duration provided from manifest is  0.71 hours.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] # 948 files loaded accounting to # 74 labels
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Found existing object /Users/gsp/.cache/torch/NeMo/NeMo_2.0.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Re-using file from: /Users/gsp/.cache/torch/NeMo/NeMo_2.0.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Instantiating model from pre-trained checkpoint


[NeMo W 2024-11-20 15:22:22 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-11-20 15:22:22 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data

[NeMo I 2024-11-20 15:22:22 nemo_logging:381] PADDING: 16
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Model EncDecSpeakerLabelModel was successfully restored from /Users/gsp/.cache/torch/NeMo/NeMo_2.0.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Dataset successfully loaded with 948 items and total duration provided from manifest is  0.71 hours.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] # 948 files loaded accounting to # 74 labels


[NeMo W 2024-11-20 15:22:22 nemo_logging:393] Total number of 74 found in all the manifest files.


[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] Dataset successfully loaded with 948 items and total duration provided from manifest is  0.71 hours.
[NeMo I 2024-11-20 15:22:22 nemo_logging:381] # 948 files loaded accounting to # 74 labels


  3%|▎         | 1/30 [03:11<1:32:30, 191.40s/it]


KeyboardInterrupt: 